In [1]:
import matplotlib.pyplot as plt

import torchvision as tv
import torch.nn.functional as F
import torch
import torch.nn as nn
from torch.autograd import Variable
import torchvision.models as models
from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import numpy as np
import torch.optim as optim
import os
import time
import loadData8
import itertools
from sklearn.metrics import confusion_matrix
import numpy as np

In [2]:
device = torch.device("cuda:0")

In [3]:
transforms = tv.transforms.Compose([
    # transforms.CenterCrop(64),
    # transforms.CenterCrop(128),
    transforms.ToTensor(),
    # transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

In [4]:
class Net8(nn.Module):
    def __init__(self):
        super(Net8, self).__init__()
        self.conv1 = nn.Conv2d(in_channels = 1, out_channels = 8, kernel_size = (1, 10))
        self.conv2 = nn.Conv2d(in_channels = 8, out_channels = 16, kernel_size = (2, 3))
        
        self.dropout1 = nn.Dropout(0.5)
        self.dropout2 = nn.Dropout(0.5)
        
        self.fc1 = nn.Linear(20304, 1024)
        self.fc2 = nn.Linear(1024, 500)
        self.fc3 = nn.Linear(500, 54)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 1)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 1)
    
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc3(x)
        output = F.log_softmax(x, dim=1)
        return output
    
net8 = Net8()
path = './model_15dB_8个0.pkl'
net8.load_state_dict(torch.load(path))
net8 = net8.to(device)

In [5]:
class Net0(nn.Module):
    def __init__(self):
        super(Net0, self).__init__()
        self.conv1 = nn.Conv2d(in_channels = 1, out_channels = 8, kernel_size = (1, 10))
        self.conv2 = nn.Conv2d(in_channels = 8, out_channels = 16, kernel_size = (2, 3))
        
        self.dropout1 = nn.Dropout(0.5)
        self.dropout2 = nn.Dropout(0.5)
        
        self.fc1 = nn.Linear(2384, 1024)
        self.fc2 = nn.Linear(1024, 500)
        self.fc3 = nn.Linear(500, 54)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 1)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 1)
    
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc3(x)
        output = F.log_softmax(x, dim=1)
        return output
    
net0 = Net0()
path = './model_15dB_1个0.pkl'
net0.load_state_dict(torch.load(path))
net0 = net0.to(device)

In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels = 1, out_channels = 8, kernel_size = (1, 10))
        self.conv2 = nn.Conv2d(in_channels = 8, out_channels = 16, kernel_size = (2, 3))
        
        self.dropout1 = nn.Dropout(0.5)
        self.dropout2 = nn.Dropout(0.5)
        
        self.fc1 = nn.Linear(10064, 1024)
        self.fc2 = nn.Linear(1024, 500)
        self.fc3 = nn.Linear(500, 54)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 1)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 1)
    
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc3(x)
        output = F.log_softmax(x, dim=1)
        return output

net = Net()
path = './model_15dB_4个0.pkl'
net.load_state_dict(torch.load(path))
net = net.to(device)

In [7]:
predict1, predict2, predict3 = [],[],[]
originLabels = []

In [8]:
import deleteFolder
for line in open("files_test15.txt"):   
    print(line)
    lines = line.split()
    locate, label = lines[0], lines[1]
    originData = np.load(locate)
    
    #测试8个0的情况
    with open("./split-no.txt" ,"w") as f8:
        f8.write(line)
    
    batchSize = 4
    test_data = loadData8.MyDataset(txt=f'split-no.txt', transform=None)
    test_loader = DataLoader(dataset=test_data, batch_size=batchSize, shuffle=True, num_workers=2)

    for data in test_loader:
        inputs, labels = data
        originLabel = int(labels)
        originLabels.append(originLabel)

        a = inputs[0:160]
#         print("---", a.shape)
        inputs, labels = inputs.cuda(), labels.cuda()
        inputs = torch.tensor(inputs, dtype=torch.float32, device = device)
        outputs = net8(inputs)
        _, predicted = torch.max(outputs.data, dim=1)
        predict1.append(int(predicted[0]))
#         print(labels,"---", predicted)
    
    
    #测试输入1个0的情况
    splitData = np.split(originData,8,axis=1)
    for i in range(len(splitData)):
#         print(splitData[i].shape)
        path1 = "/workspace/模型混合/输入1个0的中间数据/" + str(i) + ".npy"
        np.save(path1, splitData[i])
        
    with open("./split8.txt" ,"w") as f_split8:
        for i in range(len(splitData)):
            path1 = "/workspace/模型混合/输入1个0的中间数据/" + str(i) + ".npy"
            f_split8.write(f'{path1} {originLabel}\n')
            
    batchSize = 4
    test_data = loadData8.MyDataset(txt=f'split8.txt', transform=None)
    test_loader = DataLoader(dataset=test_data, batch_size=batchSize, shuffle=True, num_workers=2)
    
    
    predictedLabels = []
    for data in test_loader:
        inputs, labels = data
        inputs, labels = inputs.cuda(), labels.cuda()
        inputs = torch.tensor(inputs, dtype=torch.float32, device = device)
        outputs = net0(inputs)
        _, predicted = torch.max(outputs.data, dim=1)
        predictedLabels.append([int(label) for label in predicted])
    predictedLabels = np.reshape(predictedLabels, (1, -1))[0]
#     print("predictedLabels:", predictedLabels)
#     predict2.append(np.argmax(np.bincount(predictedLabels)))
    predict2.append([predictedLabel for predictedLabel in predictedLabels])
        
    #测试输入4个0的情况
    originData = np.load(locate)
    splitData = np.split(originData,2,axis=1)
    for i in range(len(splitData)):
#         print(splitData[i].shape)
        path1 = "/workspace/模型混合/输入4个0的中间数据/" + str(i) + ".npy"
        np.save(path1, splitData[i])
    
    with open("./split2.txt" ,"w") as f_split8:
        for i in range(len(splitData)):
            path1 = "/workspace/模型混合/输入4个0的中间数据/" + str(i) + ".npy"
            f_split8.write(f'{path1} {originLabel}\n')
    
    batchSize = 4
    test_data = loadData8.MyDataset(txt=f'split2.txt', transform=None)
    test_loader = DataLoader(dataset=test_data, batch_size=batchSize, shuffle=True, num_workers=2)
    
    
    predictedLabels = []
    for data in test_loader:
        inputs, labels = data
        inputs, labels = inputs.cuda(), labels.cuda()
        inputs = torch.tensor(inputs, dtype=torch.float32, device = device)
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, dim=1)
        predictedLabels.append([int(label) for label in predicted])
    predictedLabels = np.reshape(predictedLabels, (1, -1))[0]
#     print("predictedLabels:", predictedLabels)
    predict3.append([p for p in predictedLabels]) 
   
    
    print(originLabels[-1],  predict2[-1], predict3[-1], predict1[-1])

/workspace/时域-8个0一组的实验/15dB处理后的数据/15dBProcess_A_10_120.npy 9



/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:52: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:83: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


9 [9, 9, 10, 38, 8, 9, 25, 9] [9, 9] 9
/workspace/时域-8个0一组的实验/15dB处理后的数据/15dBProcess_A_10_121.npy 9

9 [33, 25, 9, 36, 9, 9, 10, 1] [9, 9] 9
/workspace/时域-8个0一组的实验/15dB处理后的数据/15dBProcess_A_10_122.npy 9

9 [33, 9, 9, 9, 9, 9, 38, 9] [9, 25] 9
/workspace/时域-8个0一组的实验/15dB处理后的数据/15dBProcess_A_10_123.npy 9

9 [9, 26, 36, 10, 9, 9, 10, 9] [9, 9] 9
/workspace/时域-8个0一组的实验/15dB处理后的数据/15dBProcess_A_10_124.npy 9

9 [7, 9, 9, 9, 9, 9, 10, 9] [9, 9] 9
/workspace/时域-8个0一组的实验/15dB处理后的数据/15dBProcess_A_10_125.npy 9

9 [9, 7, 9, 25, 9, 11, 9, 9] [9, 9] 9
/workspace/时域-8个0一组的实验/15dB处理后的数据/15dBProcess_A_10_126.npy 9

9 [6, 7, 9, 25, 9, 25, 9, 10] [9, 9] 9
/workspace/时域-8个0一组的实验/15dB处理后的数据/15dBProcess_A_10_127.npy 9

9 [10, 9, 33, 3, 7, 7, 38, 9] [38, 9] 38
/workspace/时域-8个0一组的实验/15dB处理后的数据/15dBProcess_A_10_128.npy 9

9 [9, 10, 25, 9, 9, 38, 25, 33] [9, 9] 9
/workspace/时域-8个0一组的实验/15dB处理后的数据/15dBProcess_A_10_129.npy 9

9 [9, 6, 9, 10, 9, 9, 9, 9] [9, 9] 9
/workspace/时域-8个0一组的实验/15dB处理后的数据/15dBProcess_A_10_

In [9]:
originLabels[0]

9

In [10]:
predict1[0]

9

In [11]:
predict2[0]

[9, 9, 10, 38, 8, 9, 25, 9]

In [12]:
predict3[0]

[9, 9]

In [13]:
print(len(originLabels), len(predict1), len(predict2), len(predict3))

1674 1674 1674 1674


In [16]:
rightNum = 0
for i in range(len(originLabels)):
    predictAll = []
    
    for _ in range(3): # 1个0一组的权重
        for a in predict2[i]:
            predictAll.append(a)
    
    for _ in range(4): # 4个0一组的权重
        for a in predict3[i]:
            predictAll.append(a)
            
    for _ in range(4): # 8个0一组的权重
        predictAll.append(predict1[i])
#     print("-----",predictAll)
#     print(p.shape)
    result = np.argmax(np.bincount(predictAll))
#     print(result, originLabels[i])
    if originLabels[i] == result:
        rightNum += 1
    else:
        print(originLabels[i], predict1[i], predict2[i], predict3[i])
print(rightNum/ len(originLabels))

9 38 [10, 9, 33, 3, 7, 7, 38, 9] [38, 9]
14 14 [15, 23, 23, 16, 23, 12, 12, 24] [14, 12]
14 14 [19, 23, 51, 23, 23, 22, 21, 18] [23, 12]
14 23 [23, 20, 23, 21, 14, 51, 23, 12] [24, 16]
14 14 [18, 12, 18, 13, 22, 23, 16, 18] [16, 14]
14 23 [49, 20, 12, 13, 23, 22, 23, 22] [23, 23]
14 14 [49, 20, 13, 24, 12, 23, 13, 12] [12, 16]
14 22 [49, 43, 23, 23, 47, 49, 12, 24] [22, 23]
14 22 [22, 22, 20, 51, 22, 23, 23, 13] [24, 19]
15 15 [44, 44, 44, 20, 44, 15, 44, 44] [15, 15]
16 16 [41, 23, 13, 41, 23, 41, 13, 23] [41, 20]
16 20 [35, 16, 16, 20, 51, 20, 20, 20] [20, 20]
16 20 [16, 14, 16, 19, 51, 20, 20, 14] [16, 20]
16 20 [14, 16, 20, 20, 16, 14, 16, 18] [20, 16]
16 16 [20, 14, 23, 51, 18, 14, 14, 14] [16, 16]
16 20 [16, 20, 20, 20, 21, 20, 19, 43] [16, 16]
18 22 [18, 40, 19, 22, 49, 22, 21, 22] [18, 18]
18 19 [47, 47, 19, 19, 18, 49, 19, 41] [20, 19]
19 22 [21, 20, 47, 49, 47, 47, 19, 47] [19, 19]
19 47 [18, 18, 22, 18, 18, 21, 19, 22] [19, 19]
20 43 [47, 20, 41, 21, 20, 41, 41, 41] [20, 41]

In [15]:
# 加权值
